In [ ]:
# Requirements
!pip install nibabel vtk

In [ ]:
import k3d
import numpy as np
from k3d.helpers import download

In [ ]:
import vtk
from vtk.util import numpy_support

filename = download('https://vedo.embl.es/examples/data/embryo.slc')
reader = vtk.vtkSLCReader()
reader.SetFileName(filename)
reader.Update()
vti = reader.GetOutput()

bounds = vti.GetBounds()
x, y, z = vti.GetDimensions()
volume_data = numpy_support.vtk_to_numpy(vti.GetPointData().GetArray(0)).reshape(-1, y, x)

In [ ]:
embryo = k3d.volume_slice(volume_data.astype(np.float16), 
                          color_map=np.array(k3d.basic_color_maps.BlackBodyRadiation, dtype=np.float32), 
                          slice_z=volume_data.shape[0]//2,
                          slice_y=volume_data.shape[1]//2,
                          slice_x=volume_data.shape[2]//2,
                          bounds=bounds)

plot = k3d.plot(camera_mode='volume_sides', grid_visible=False, background_color=0)
plot += embryo
plot.display()

In [ ]:
with open('binary_snapshot.k3d', 'wb') as f:
    f.write(plot.get_binary_snapshot())

In [ ]:
mask = np.empty_like(volume_data, dtype=np.uint8)
mask[:, :mask.shape[1]//2, :] = 1
mask[:, mask.shape[1]//2:, :] = 2

In [ ]:
embryo = k3d.volume_slice(volume_data.astype(np.float16), 
                          color_map=np.array(k3d.paraview_color_maps.Grayscale, dtype=np.float32), 
                          slice_z=volume_data.shape[0]//2,
                          slice_y=volume_data.shape[1]//2,
                          slice_x=volume_data.shape[2]//2,
                          active_masks=[1,2],
                          mask=mask,
                          mask_opacity=0.5,
                          bounds=bounds)

plot = k3d.plot(camera_mode='volume_sides', grid_visible=False, background_color=0)
plot += embryo
plot.display()